# [과제 3] 로지스틱 회귀분석
### - sklearn 패키지를 사용해 로지스틱 회귀분석을 진행해주세요.
### - 성능지표를 계산하고 이에 대해 해석해주세요.
### - 성능 개선을 시도해주세요. (어떠한 성능지표를 기준으로 개선을 시도했는지, 그 이유도 함께 적어주세요.)
### - 주석으로 설명 및 근거 자세하게 달아주시면 감사하겠습니다. :)

## Data 

출처 : https://www.kaggle.com/mlg-ulb/creditcardfraud


* V1 ~ V28 : 비식별화 된 개인정보 
* **Class** : Target 변수  
  - 1 : fraudulent transactions (사기)
  - 0 : otherwise 

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
data = pd.read_csv("assignment3_creditcard.csv")

In [5]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,-1.848212,2.384900,0.379573,1.048381,-0.845070,2.537837,-4.542983,-10.201458,-1.504967,-2.234167,...,2.585817,-5.291690,0.859364,0.423231,-0.506985,1.020052,-0.627751,-0.017753,0.280982,0
1,2.071805,-0.477943,-1.444444,-0.548657,0.010036,-0.582242,-0.042878,-0.247160,1.171923,-0.342382,...,-0.077306,0.042858,0.390125,0.041569,0.598427,0.098803,0.979686,-0.093244,-0.065615,0
2,-2.985294,-2.747472,1.194068,-0.003036,-1.151041,-0.263559,0.553500,0.635600,0.438545,-1.806488,...,1.345776,0.373760,-0.385777,1.197596,0.407229,0.008013,0.762362,-0.299024,-0.303929,0
3,-1.479452,1.542874,0.290895,0.838142,-0.529290,-0.717661,0.484516,0.545092,-0.780767,0.324804,...,0.038397,0.116771,0.405560,-0.116453,0.541275,-0.216665,-0.415578,0.027126,-0.150347,0
4,-0.281976,-0.309699,-2.162299,-0.851514,0.106167,-1.483888,1.930994,-0.843049,-1.249272,1.079608,...,-0.875516,-0.004199,1.015108,-0.026748,0.077115,-1.468822,0.751700,0.496732,0.331001,0


In [6]:
data.info()
#결측값 없음 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28678 entries, 0 to 28677
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      28678 non-null  float64
 1   V2      28678 non-null  float64
 2   V3      28678 non-null  float64
 3   V4      28678 non-null  float64
 4   V5      28678 non-null  float64
 5   V6      28678 non-null  float64
 6   V7      28678 non-null  float64
 7   V8      28678 non-null  float64
 8   V9      28678 non-null  float64
 9   V10     28678 non-null  float64
 10  V11     28678 non-null  float64
 11  V12     28678 non-null  float64
 12  V13     28678 non-null  float64
 13  V14     28678 non-null  float64
 14  V15     28678 non-null  float64
 15  V16     28678 non-null  float64
 16  V17     28678 non-null  float64
 17  V18     28678 non-null  float64
 18  V19     28678 non-null  float64
 19  V20     28678 non-null  float64
 20  V21     28678 non-null  float64
 21  V22     28678 non-null  float64
 22

#상관관계

In [7]:
#다 이산형변수이므로 상관관계를 분석해보자
corr=data[['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23',
      'V24','V25','V26','V27','V28']].corr()
corr

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
V1,1.000000,-0.201825,0.163547,-0.069693,0.157502,0.001112,0.200443,-0.020877,0.086285,0.163211,...,-0.020525,-0.074624,-0.048482,0.011011,0.039224,0.001055,0.007906,0.007325,0.014890,0.022776
V2,-0.201825,1.000000,-0.168389,0.125243,-0.068987,-0.047479,-0.154898,-0.011445,-0.061759,-0.136513,...,0.033629,-0.077703,0.017631,-0.012426,0.079599,-0.004122,0.040946,0.014690,0.018948,0.017515
V3,0.163547,-0.168389,1.000000,-0.132964,0.178195,0.050228,0.269471,-0.041581,0.125052,0.250415,...,-0.035101,-0.046755,-0.031438,0.006083,-0.001562,0.008052,-0.009167,-0.006336,-0.004618,0.017944
V4,-0.069693,0.125243,-0.132964,1.000000,-0.078587,-0.022494,-0.146943,0.026149,-0.082091,-0.142944,...,0.024533,0.038922,0.022779,0.002485,-0.007806,0.002472,-0.003070,0.006063,0.006666,-0.002075
V5,0.157502,-0.068987,0.178195,-0.078587,1.000000,0.024967,0.180761,-0.042485,0.071119,0.146717,...,-0.039411,0.003591,-0.008040,0.003934,-0.040210,-0.002584,-0.022166,-0.005083,-0.016251,0.030842
V6,0.001112,-0.047479,0.050228,-0.022494,0.024967,1.000000,0.042292,-0.040344,0.032544,0.054229,...,-0.009077,-0.014122,0.013126,-0.009390,0.038604,0.008377,-0.001263,-0.000455,0.000930,-0.031265
V7,0.200443,-0.154898,0.269471,-0.146943,0.180761,0.042292,1.000000,0.026257,0.120892,0.243324,...,-0.050808,-0.015847,-0.036292,0.000570,-0.015004,-0.000209,-0.015047,-0.005232,0.025076,0.000575
V8,-0.020877,-0.011445,-0.041581,0.026149,-0.042485,-0.040344,0.026257,1.000000,-0.015805,-0.004709,...,0.021175,0.005671,-0.118040,0.037955,-0.050580,0.012656,0.015667,-0.002378,0.024665,0.023833
V9,0.086285,-0.061759,0.125052,-0.082091,0.071119,0.032544,0.120892,-0.015805,1.000000,0.109749,...,-0.018153,-0.018130,0.017662,-0.015938,-0.007020,0.003328,-0.004485,-0.009288,0.010659,-0.008113
V10,0.163211,-0.136513,0.250415,-0.142944,0.146717,0.054229,0.243324,-0.004709,0.109749,1.000000,...,-0.052385,-0.019257,0.005864,-0.023205,-0.015780,0.010106,0.002799,0.001433,0.010375,0.029260


In [8]:
corr.describe()
#다중공산성은 없는듯 하다. 

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
count,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,...,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000
mean,0.072437,-0.002770,0.089823,-0.002784,0.073437,0.046664,0.094816,0.025789,0.067753,0.097900,...,0.020934,0.027369,0.021668,0.030091,0.031438,0.037438,0.034241,0.034208,0.037137,0.048551
std,0.204602,0.214182,0.219896,0.212729,0.200788,0.189638,0.221479,0.194073,0.195754,0.222830,...,0.194624,0.195143,0.197469,0.191558,0.192218,0.188804,0.189851,0.189417,0.191379,0.189433
min,-0.201825,-0.201825,-0.168389,-0.173920,-0.086212,-0.047479,-0.157529,-0.118040,-0.087017,-0.180378,...,-0.089063,-0.105791,-0.118040,-0.101605,-0.051476,-0.014703,-0.050416,-0.015733,-0.051476,-0.031265
25%,-0.004340,-0.108232,-0.014735,-0.100849,-0.017729,-0.009155,-0.015015,-0.032954,-0.010917,-0.009631,...,-0.037862,-0.032898,-0.036920,-0.009714,-0.016563,-0.002673,-0.007370,-0.005508,-0.014871,-0.007576
50%,0.018833,-0.011936,0.007067,-0.005438,0.003763,0.006540,0.013135,-0.003544,0.008632,0.010241,...,-0.008385,-0.001846,-0.001200,-0.002783,-0.006197,0.001798,0.001325,-0.000314,-0.001581,0.001136
75%,0.129627,0.017960,0.182553,0.010694,0.128588,0.043097,0.204384,0.021740,0.109875,0.184876,...,0.007093,0.015364,0.017639,0.004472,0.004073,0.008133,0.004209,0.004429,0.015904,0.025189
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#종속변수 value_counts()

In [9]:
data['Class'].value_counts()
#굉장히 불균형 데이터이다.

0    28432
1      246
Name: Class, dtype: int64

#모델링 

In [10]:
x_data=data[['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23',
      'V24','V25','V26','V27','V28'
             ]].to_numpy() #전체 독립변수 일단 다 사용하기 
y_data=data[['Class']].to_numpy()

In [11]:
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target=train_test_split(x_data,y_data,test_size=0.2,random_state=42) #훈련세트와 테스트세트 분리 
print(train_input.shape,train_target.shape)
print(test_input.shape,test_target.shape)

(22942, 28) (22942, 1)
(5736, 28) (5736, 1)


In [12]:
from sklearn.preprocessing import StandardScaler #스탠다드 스케일링 사용 
ss=StandardScaler()
ss.fit(train_input)
train_scaled=ss.transform(train_input)
test_scaled=ss.transform(test_input)
print(train_scaled.shape,test_scaled.shape)

(22942, 28) (5736, 28)


In [13]:
#불균형데이터이므로 오버샘플링을 진행하려고 하였으나
#차이가 너무 심해 가상의 데이터를 너무 많이 만들어낸다고 생각해서 그냥 진행
'''
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
train_scaled_over, train_target_over = smote.fit_resample(train_scaled, train_target)
print("SMOTE 적용 전 학습용 피처/레이블 데이터 세트 : ", train_scaled.shape, train_target.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트 :', train_scaled_over.shape, train_target_over.shape)
'''

'\nfrom imblearn.over_sampling import SMOTE\n\nsmote = SMOTE(random_state=42)\ntrain_scaled_over, train_target_over = smote.fit_resample(train_scaled, train_target)\nprint("SMOTE 적용 전 학습용 피처/레이블 데이터 세트 : ", train_scaled.shape, train_target.shape)\nprint(\'SMOTE 적용 후 학습용 피처/레이블 데이터 세트 :\', train_scaled_over.shape, train_target_over.shape)\n'

In [14]:
from sklearn.linear_model import LogisticRegression 
logR=LogisticRegression() #로지스틱회귀함수 객체 생성 
logR.fit(train_scaled,train_target)


LogisticRegression()

In [15]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score #혼동행렬, 정확도, 정밀도, 재현율,f1_score 함수 import
pred = logR.predict(test_scaled) 
test_acc=accuracy_score(test_target.ravel(), pred) 
test_cm=confusion_matrix(test_target.ravel(),pred)
test_prc=precision_score(test_target.ravel(),pred)
test_rcll=precision_score(test_target.ravel(),pred)
test_f1=f1_score(test_target.ravel(),pred)

print(test_cm)
print('\n')
print('정확도:',round(test_acc*100,2))
print('정밀도:',round(test_prc*100,2)) 
print('재현율:',round(test_rcll*100,2))  
print('F1:',round(test_f1*100,2)) 
#불균형데이터이므로 정확도, 정밀도, 재현율은 별로 의미가 없다. F1스코어를 올리기 위해 여러시도를 해보았다. 
#0의 class 개수가 훨씬 많기 때문에 TP또는 TN의 값이 굉장히 클 수 밖에 없다. 따라서 불균형 데이터의 경우 정확도는 항상 크게 나오고 , 정밀도와 재현율은 엄청 작게 또는 엄청 크게 나올 수 밖에 없다. 
#위의 데이터의 경우 0 class가 positive로 분류되어 정확도, 정밀도, 재현율이 모두 매우 큰값으로 나왔다. 
#따라서 성능개선을 위해서는 F1_score을 올려야 한다. 

[[5688    0]
 [   7   41]]


정확도: 99.88
정밀도: 100.0
재현율: 100.0
F1: 92.13


#로지스틱회귀 하이퍼파라미터조절 

In [16]:
from sklearn.linear_model import LogisticRegression 
logR=LogisticRegression()
logR.fit(train_scaled,train_target)

LogisticRegression()

In [17]:
print(logR.get_params().keys())

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])


In [19]:
from sklearn.model_selection import GridSearchCV #교차검증과 하이퍼파라미터조절을 같이해주는 그리드서치를 이용함. 
params={'max_iter':[100,300,500],'C':[0.01,0.1,1,5,10]}
gs=GridSearchCV(logR,params,n_jobs=-1,cv=3,scoring='roc_auc',verbose=1) #재현율의 성능을 높이는 것으로 기준 정확도는 이미 높다고 생각. 
gs.fit(train_scaled,train_target)
dt=gs.best_params_
print(dt) #최고의 파라미터 출력 
print(gs.best_score_)
#최적의 파라미터는 c:10,max_iter=100
#roc_auc 0.9857까지 올릴수 있었음. 

Fitting 3 folds for each of 15 candidates, totalling 45 fits
{'C': 0.1, 'max_iter': 100}
0.9771151056689558


#변수선택+오버샘플링 

In [33]:
x_data=data.loc[:,['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23',
      'V24','V25','V26','V27','V28'
             ]]
y_data=data.loc[:,['Class']]

In [34]:
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target=train_test_split(x_data,y_data,test_size=0.2,random_state=42)
print(train_input.shape,train_target.shape)
print(test_input.shape,test_target.shape)

(22942, 28) (22942, 1)
(5736, 28) (5736, 1)


In [35]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
ss.fit(train_input)
train_scaled=ss.transform(train_input)
test_scaled=ss.transform(test_input)
print(train_scaled.shape,test_scaled.shape)

(22942, 28) (5736, 28)


In [36]:
#이번에는 클래스 불균형을 해결하기 위해서 오버샘플링을 진행하자. 
#다운 샘플링을 하면 데이터의 개수가 많이 없어지므로 오버샘플링을 진행하였다. 
#오버샘플링 중에서도 SMOTE 기법을 이용하였다. 
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
train_scaled_over, train_target_over = smote.fit_resample(train_scaled, train_target)
print("SMOTE 적용 전 학습용 피처/레이블 데이터 세트 : ", train_scaled.shape, train_target.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트 :', train_scaled_over.shape, train_target_over.shape)


SMOTE 적용 전 학습용 피처/레이블 데이터 세트 :  (22942, 28) (22942, 1)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트 : (45488, 28) (45488, 1)


In [37]:
from sklearn.linear_model import LogisticRegression 
logR=LogisticRegression()
logR.fit(train_scaled_over,train_target_over)


LogisticRegression()

In [40]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score
pred = logR.predict(test_scaled) 
test_acc=accuracy_score(test_target, pred) 
test_cm=confusion_matrix(test_target,pred)
test_prc=precision_score(test_target,pred)
test_rcll=precision_score(test_target,pred)
test_f1=f1_score(test_target,pred)

print(test_cm)
print('\n')
print('정확도:',round(test_acc*100,2))
print('정밀도:',round(test_prc*100,2)) 
print('재현율:',round(test_rcll*100,2))  
print('F1:',round(test_f1*100,2)) 
#오버샘플링을 하니 확실히 모든 지표의 값이 내려간 것을 확인할 수 있다. 전반적인 지표의 성능을 높혀보자. 

[[5566  122]
 [   4   44]]


정확도: 97.8
정밀도: 26.51
재현율: 26.51
F1: 41.12


In [41]:
#변수 선택을 위해 "변수 중요도"를 알려주는 모델에 넣은뒤 변수중요도를 출력해보자
#모델로는 랜덤 포레스트를 사용하였다. 
!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score
rfc = RandomForestClassifier(n_estimators=100,min_samples_split=5,criterion='gini')
rfc.fit(train_scaled_over , train_target_over)
pred = rfc.predict(test_scaled)
test_acc=accuracy_score(test_target, pred) 
print(test_acc)

0.9987796373779637


In [43]:
import eli5
from eli5.sklearn import PermutationImportance
perm=PermutationImportance(rfc,random_state=42).fit(test_scaled,test_target)

eli5.show_weights(perm,feature_names=train_input.columns.tolist())

Weight,Feature
0.0016 ± 0.0010,V12
0.0005 ± 0.0002,V14
0.0001 ± 0.0003,V15
0.0001 ± 0.0002,V19
0.0000 ± 0.0003,V8
0 ± 0.0000,V2
0 ± 0.0000,V23
0 ± 0.0000,V21
-0.0000 ± 0.0002,V4
-0.0000 ± 0.0002,V28


In [56]:
x_data_ch=data[['V12','V14','V15','V19','V8','V2','V23','V21','V4','V28','V10','V18'
             ]].to_numpy() #독립변수의 개수를 줄여보았다. 
y_data=data[['Class']].to_numpy()

In [57]:
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target=train_test_split(x_data_ch,y_data,test_size=0.2,random_state=42)
print(train_input.shape,train_target.shape)
print(test_input.shape,test_target.shape)

(22942, 12) (22942, 1)
(5736, 12) (5736, 1)


In [58]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
ss.fit(train_input)
train_scaled=ss.transform(train_input)
test_scaled=ss.transform(test_input)
print(train_scaled.shape,test_scaled.shape)

(22942, 12) (5736, 12)


In [59]:
#불균형데이터이므로 오버샘플링을 진행하려고 하였으나
#차이가 너무 심해 가상의 데이터를 너무 많이 만들어낸다고 생각해서 그냥 진행

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
train_scaled_over, train_target_over = smote.fit_resample(train_scaled, train_target)
print("SMOTE 적용 전 학습용 피처/레이블 데이터 세트 : ", train_scaled.shape, train_target.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트 :', train_scaled_over.shape, train_target_over.shape)


SMOTE 적용 전 학습용 피처/레이블 데이터 세트 :  (22942, 12) (22942, 1)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트 : (45488, 12) (45488,)


In [60]:
from sklearn.linear_model import LogisticRegression 
logR=LogisticRegression(C=10,max_iter=100) #아까 하이퍼파라미터의 결과 최적의 파라미터 조합이다. 
logR.fit(train_scaled_over,train_target_over)


LogisticRegression(C=10)

In [61]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score
pred = logR.predict(test_scaled) 
test_acc=accuracy_score(test_target.ravel(), pred) 
test_cm=confusion_matrix(test_target.ravel(),pred)
test_prc=precision_score(test_target.ravel(),pred)
test_rcll=precision_score(test_target.ravel(),pred)
test_f1=f1_score(test_target.ravel(),pred)

print(test_cm)
print('\n')
print('정확도:',round(test_acc*100,2))
print('정밀도:',round(test_prc*100,2)) 
print('재현율:',round(test_rcll*100,2))  
print('F1:',round(test_f1*100,2)) 
#모든 변수를 다 선택하여 오버샘플링 + 로지스틱 회귀 결과 
#정확도 : 97.8
#정밀도 : 26.51
#재현율 : 26.51
#F1 : 41.12
#모든 성과지표의 성능이 오름을 확인할 수 있다. 

[[5569  119]
 [   3   45]]


정확도: 97.87
정밀도: 27.44
재현율: 27.44
F1: 42.45
